# Overview

TraceCalls.jl is a functional tracing package, for debugging and exploring Julia code. It records and displays a tree of function calls. For example, here is how [Calculus.jl](https://github.com/johnmyleswhite/Calculus.jl) computes the second-derivative:

In [1]:
using Calculus, TraceCalls

@traceable f(x) = sin(x) + sqrt(x)
trace_derivative = @trace Calculus second_derivative(f, 1.0)

INFO: Recompiling stale cache file /Users/cedric/.julia/lib/v0.6/TraceCalls.ji for module TraceCalls.


TraceCalls.Trace(#1, (), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.second_derivative, (f, 1.0), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.derivative, (f, :central), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.derivative, (f, :scalar, :central), (), TraceCalls.Trace[], Calculus.#25)], Calculus.#25), TraceCalls.Trace(Calculus.finite_difference_hessian, (f, Calculus.#25, 1.0, :central), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.finite_difference, (Calculus.#25, 1.0, :central), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.finite_difference, (f, 1.00001, :central), (), TraceCalls.Trace[TraceCalls.Trace(f, (1.00001,), (), TraceCalls.Trace[], 1.84148), TraceCalls.Trace(f, (1.0,), (), TraceCalls.Trace[], 1.84147)], 1.0403), TraceCalls.Trace(Calculus.finite_difference, (f, 0.999994, :central), (), TraceCalls.Trace[TraceCalls.Trace(f, (1.0,), (), TraceCalls.Trace[], 1.84147), TraceCalls.Trace(f, (0.999988,), (), TraceCalls.Trace[], 1.84146)], 1.04031)], -1.09147)], -1.09147)], -1.09147)], -1.0914708926553454)

The output of `@trace` is a `Trace` object --- an explorable tree-like datastructure:

In [2]:
trace_derivative[1][1]          # get the first call of the first call. Can also be written `trace_derivative[1, 1]`

TraceCalls.Trace(Calculus.derivative, (f, :central), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.derivative, (f, :scalar, :central), (), TraceCalls.Trace[], Calculus.#25)], Calculus.#25)

In [3]:
trace_derivative[1][1].args[2]  # get its second argument

:central

It can work as a more-informative stack-trace (which contains _values_ rather than just types - see `@stacktrace` below):

In [4]:
@stacktrace Calculus second_derivative(f, 0.0)

TraceCalls.Trace(#3, (), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.second_derivative, (f, 0.0), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.finite_difference_hessian, (f, Calculus.#63, 0.0, :central), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.finite_difference, (Calculus.#63, 0.0, :central), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.finite_difference, (f, -6.05545e-6, :central), (), TraceCalls.Trace[TraceCalls.Trace(f, (-1.21109e-5,), (), TraceCalls.Trace[], DomainError())], DomainError())], DomainError())], DomainError())], DomainError())], DomainError())

In [5]:
@trace Calculus second_derivative(f, 0.0)

TraceCalls.Trace(#5, (), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.second_derivative, (f, 0.0), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.derivative, (f, :central), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.derivative, (f, :scalar, :central), (), TraceCalls.Trace[], Calculus.#101)], Calculus.#101), TraceCalls.Trace(Calculus.finite_difference_hessian, (f, Calculus.#101, 0.0, :central), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.finite_difference, (Calculus.#101, 0.0, :central), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.finite_difference, (f, 6.05545e-6, :central), (), TraceCalls.Trace[TraceCalls.Trace(f, (1.21109e-5,), (), TraceCalls.Trace[], 0.00349218), TraceCalls.Trace(f, (0.0,), (), TraceCalls.Trace[], 0.0)], 288.35), TraceCalls.Trace(Calculus.finite_difference, (f, -6.05545e-6, :central), (), TraceCalls.Trace[TraceCalls.Trace(f, (0.0,), (), TraceCalls.Trace[], 0.0), TraceCalls.Trace(f, (-1.21109e-5,), (), TraceCalls.Trace[], DomainError())], DomainError())], DomainError())], DomainError())], DomainError())], DomainError())

And finally, since full call data is recorded, we can rerun every part of the trace for profiling, debugging, or testing.

In [6]:
greenred(map(:@allocated, trace_derivative))    # compute how many bytes were allocated in each function call

TraceCalls.Trace(#1, (), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.second_derivative, (f, 1.0), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.derivative, (f, :central), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.derivative, (f, :scalar, :central), (), TraceCalls.Trace[], TraceCalls.FontColor("ff0000", 42200))], TraceCalls.FontColor("ff0000", 208)), TraceCalls.Trace(Calculus.finite_difference_hessian, (f, Calculus.#25, 1.0, :central), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.finite_difference, (Calculus.#25, 1.0, :central), (), TraceCalls.Trace[TraceCalls.Trace(Calculus.finite_difference, (f, 1.00001, :central), (), TraceCalls.Trace[TraceCalls.Trace(f, (1.00001,), (), TraceCalls.Trace[], TraceCalls.FontColor("00ff00", 0)), TraceCalls.Trace(f, (1.0,), (), TraceCalls.Trace[], TraceCalls.FontColor("00ff00", 0))], TraceCalls.FontColor("00ff00", 0)), TraceCalls.Trace(Calculus.finite_difference, (f, 0.999994, :central), (), TraceCalls.Trace[TraceCalls.Trace(f, (1.0,), (), TraceCalls.Trace[], TraceCalls.FontColor("00ff00", 0)), TraceCalls.Trace(f, (0.999988,), (), TraceCalls.Trace[], TraceCalls.FontColor("00ff00", 0))], TraceCalls.FontColor("00ff00", 0))], TraceCalls.FontColor("ff0000", 16))], TraceCalls.FontColor("ff0000", 16))], TraceCalls.FontColor("ff0000", 419023))], TraceCalls.FontColor("ff0000", 256))

# Tracing code

The best way to trace your own codebase is to annotate key functions with `@traceable`.

In [7]:
@traceable function my_factorial(n)
    if n==0
        return 1
    else
        return n * my_factorial(n-1)
    end
end

# How many ways are there of selecting k amongst n. See https://en.wikipedia.org/wiki/Combination
combination(k, n) = my_factorial(n) // (my_factorial(n-k) * my_factorial(k))

@trace combination(2, 3)

TraceCalls.Trace(#19, (), (), TraceCalls.Trace[TraceCalls.Trace(my_factorial, (3,), (), TraceCalls.Trace[TraceCalls.Trace(my_factorial, (2,), (), TraceCalls.Trace[TraceCalls.Trace(my_factorial, (1,), (), TraceCalls.Trace[TraceCalls.Trace(my_factorial, (0,), (), TraceCalls.Trace[], 1)], 1)], 2)], 6), TraceCalls.Trace(my_factorial, (1,), (), TraceCalls.Trace[TraceCalls.Trace(my_factorial, (0,), (), TraceCalls.Trace[], 1)], 1), TraceCalls.Trace(my_factorial, (2,), (), TraceCalls.Trace[TraceCalls.Trace(my_factorial, (1,), (), TraceCalls.Trace[TraceCalls.Trace(my_factorial, (0,), (), TraceCalls.Trace[], 1)], 1)], 2)], 3//1)

In the example above, `my_factorial` is traced (each call to it will be recorded when `@trace` is used), but `combination` isn't. Crucially, the `@traceable` macro merely remembers the function definition, it does not modify it in any way, which means that **@traceable has zero impact on the performance of your code**. Sprinkle it liberally!

Sometimes, it's not practical or convenient to add `@traceable` annotations. You can trace entire modules and specific functions by passing them to the `@trace` macro:

In [8]:
using DataStructures
to_trace = (DataStructures.nextreme, DataStructures.compare)
@trace to_trace nlargest(3, [0,21,-12,68,-25,14])  # returns the 3 largest values in the vector

TraceCalls.Trace(#21, (), (), TraceCalls.Trace[TraceCalls.Trace(DataStructures.nextreme, (DataStructures.LessThan(), 3, [0, 21, -12, 68, -25, 14]), (), TraceCalls.Trace[TraceCalls.Trace(DataStructures.compare, (DataStructures.LessThan(), 21, 0), (), TraceCalls.Trace[], false), TraceCalls.Trace(DataStructures.compare, (DataStructures.LessThan(), -12, 0), (), TraceCalls.Trace[], true), TraceCalls.Trace(DataStructures.compare, (DataStructures.LessThan(), -12, 68), (), TraceCalls.Trace[], true), TraceCalls.Trace(DataStructures.compare, (DataStructures.LessThan(), 21, 0), (), TraceCalls.Trace[], false), TraceCalls.Trace(DataStructures.compare, (DataStructures.LessThan(), 68, 0), (), TraceCalls.Trace[], false), TraceCalls.Trace(DataStructures.compare, (DataStructures.LessThan(), 0, -25), (), TraceCalls.Trace[], false), TraceCalls.Trace(DataStructures.compare, (DataStructures.LessThan(), 0, 14), (), TraceCalls.Trace[], true), TraceCalls.Trace(DataStructures.compare, (DataStructures.LessThan(), 21, 68), (), TraceCalls.Trace[], true), TraceCalls.Trace(DataStructures.compare, (DataStructures.LessThan(), 14, 21), (), TraceCalls.Trace[], true), TraceCalls.Trace(DataStructures.compare, (DataStructures.LessThan(), 68, 21), (), TraceCalls.Trace[], false)], [68, 21, 14])], [68, 21, 14])

`@trace` attempts to find the source of every method of the traced functions, and emits a warning (see above) when it is missing some. When tracing modules, `@trace` has to parse their source code looking for function definitions (using [Revise.jl](https://github.com/timholy/Revise.jl)). That is an inexact science; many functions will not be traced. In particular, `@trace` and `@traceable` ignore:

 - Inner constructors
 - Functions that are defined by `eval`
 - Function definitions inside a macro (eg. `@inline foo(x) = x+2`), unless `@traceable` is part of its function definition.
 

#### Syntax

The full syntax of `@trace` is:

    @trace (function1, function2, ..., module1, module2, ...) code_to_execute

It will trace the given functions, the functions inside the given modules, and all functions that were defined with `@traceable`. The shorthand `@trace code_to_trace` is equivalent to `@trace () code_to_trace`.

#### Implementation of @trace

`@trace some_fn foo(x)` performs these operations:

1. Replace the definition(s) of `some_fn` with a tracing version (using `eval`)
2. Run foo(x)
3. Restore the original definition of `some_fn` (using `eval`)

The downside of this scheme is that `@trace` will have to compile every traced function twice. Tracing large modules can be slow the first time around, but caching is used to avoid repeated computations.

# Manipulating traces

Consider computing a [random walk](https://en.wikipedia.org/wiki/Random_walk) on a small graph, using [LightGraphs.jl](http://juliagraphs.github.io/LightGraphs.jl/latest/)

In [9]:
using LightGraphs

graph = Graph(3)                                      # build an undirected graph with three connected vertices
add_edge!(graph, 1, 2); add_edge!(graph, 2, 3) 

trace_walk = @trace LightGraphs randomwalk(graph, 2, 5)

TraceCalls.Trace(#23, (), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.randomwalk, ({3, 2} undirected simple Int64 graph, 2, 5), (), TraceCalls.Trace[TraceCalls.Trace(eltype, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[], Int64), TraceCalls.Trace(LightGraphs.vertices, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.nv, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[TraceCalls.Trace(eltype, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[], Int64), TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[], Array{Int64,1}[[2], [1, 3], [2]])], 3)], 1:3), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], [1, 3])], [1, 3]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 1), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph, 1), (), TraceCalls.Trace[], [2])], [2]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], [1, 3])], [1, 3]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 1), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph, 1), (), TraceCalls.Trace[], [2])], [2]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], [1, 3])], [1, 3])], [2, 1, 2, 1, 2])], [2, 1, 2, 1, 2])

The trace can be indexed:

In [24]:
trace_walk[1][3]    # can also be written graph_trace[1,3]

TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], [1, 3])], [1, 3])

Called:

In [25]:
trace_walk[1,3]()   # call `LightGraphs.out_neighbors({2, 1} directed simple Int64 graph, 2)`

2-element Array{Int64,1}:
 1
 3

Or pruned (very useful for exploring large traces):

In [12]:
prune(trace_walk, 
    2,  # maximum depth
    5)  # maximum length of each trace (eg. if foo() calls bar() 100 times)

TraceCalls.Trace(#23, (), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.randomwalk, ({3, 2} undirected simple Int64 graph, 2, 5), (), TraceCalls.Trace[TraceCalls.Trace(eltype, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[], Int64), TraceCalls.Trace(LightGraphs.vertices, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[], 1:3), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], [1, 3]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 1), (), TraceCalls.Trace[], [2]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], [1, 3])], [2, 1, 2, 1, 2])], [2, 1, 2, 1, 2])

### Custom HTML

To display each argument and return value, `TraceCalls.val_html(x)` is called. It defaults to `repr(x)`, but can be customized, either to highlight certain values, or to shorten values that are not important for the task at hand.

In [13]:
TraceCalls.val_html(v::Vector{Int}) = string("[", join([x==2 ? "<font color=red>2</font>" : x for x in v], ","), "]")
TraceCalls.val_html(::Graph) = "AnyOldGraph"
trace_walk

TraceCalls.Trace(#23, (), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.randomwalk, ({3, 2} undirected simple Int64 graph, 2, 5), (), TraceCalls.Trace[TraceCalls.Trace(eltype, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[], Int64), TraceCalls.Trace(LightGraphs.vertices, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.nv, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[TraceCalls.Trace(eltype, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[], Int64), TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph,), (), TraceCalls.Trace[], Array{Int64,1}[[2], [1, 3], [2]])], 3)], 1:3), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], [1, 3])], [1, 3]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 1), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph, 1), (), TraceCalls.Trace[], [2])], [2]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], [1, 3])], [1, 3]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 1), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph, 1), (), TraceCalls.Trace[], [2])], [2]), TraceCalls.Trace(LightGraphs.out_neighbors, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[TraceCalls.Trace(LightGraphs.SimpleGraphs.fadj, ({3, 2} undirected simple Int64 graph, 2), (), TraceCalls.Trace[], [1, 3])], [1, 3])], [2, 1, 2, 1, 2])], [2, 1, 2, 1, 2])

See also `?TraceCalls.call_html` and `?TraceCalls.return_val_html`

### Map, filter, collect

```julia
struct Trace
    func                  # the function called
    args::Tuple           # the positional arguments
    kwargs::Tuple         # the keyword arguments
    called::Vector{Trace} # the functions called within the execution of this function call 
    value                 # This is the return value of the func(args...; kwargs...) call, but it's also where
                          # the result of `map(f, ::Trace)` will be stored.
end
```

# Working with mutable state

Because `@trace` stores the function arguments without copying them, tracing function calls that modify their arguments can yield surprising results. Consider this trivial example, that generates a vector of `n` 5s.

In [14]:
@traceable push5!(vec::Vector) = push!(vec, 5)
@traceable function many_5s(n)
    vec = Int[]
    for i in 1:n
        push5!(vec)
    end
    return vec
end

@trace many_5s(3)

TraceCalls.Trace(#27, (), (), TraceCalls.Trace[TraceCalls.Trace(many_5s, (3,), (), TraceCalls.Trace[TraceCalls.Trace(push5!, ([5, 5, 5],), (), TraceCalls.Trace[], [5, 5, 5]), TraceCalls.Trace(push5!, ([5, 5, 5],), (), TraceCalls.Trace[], [5, 5, 5]), TraceCalls.Trace(push5!, ([5, 5, 5],), (), TraceCalls.Trace[], [5, 5, 5])], [5, 5, 5])], [5, 5, 5])

When `push5!` was first called, `vec` was empty, but this trace makes it look like it already had three 5s in it. This is because all vectors in that trace are [the same object](http://www.johnmyleswhite.com/notebook/2014/09/06/values-vs-bindings-the-map-is-not-the-territory/). As long as you keep this peculiarity in mind, it is not necessarily a problem. But if you do care, here are a few solutions. The simplest is to take the mutating functions out of the trace. This is a good option for profiling.

In [15]:
tr = @trace many_5s(3)
filter(!is_mutating, tr)    # filter out every function that ends with a ! (see https://docs.julialang.org/en/stable/manual/style-guide/#Append-!-to-names-of-functions-that-modify-their-arguments-1)

TraceCalls.Trace(#29, (), (), TraceCalls.Trace[TraceCalls.Trace(many_5s, (3,), (), TraceCalls.Trace[], [5, 5, 5])], [5, 5, 5])

Alternatively, you can tell TraceCalls to make a copy of each `Vector` argument when storing it:

In [16]:
TraceCalls.store(x::Vector) = copy(x)
@trace many_5s(3)

TraceCalls.Trace(#31, (), (), TraceCalls.Trace[TraceCalls.Trace(many_5s, (3,), (), TraceCalls.Trace[TraceCalls.Trace(push5!, (Int64[],), (), TraceCalls.Trace[], [5]), TraceCalls.Trace(push5!, ([5],), (), TraceCalls.Trace[], [5, 5]), TraceCalls.Trace(push5!, ([5, 5],), (), TraceCalls.Trace[], [5, 5, 5])], [5, 5, 5])], [5, 5, 5])

The most drastic solution is to have TraceCalls store the HTML representation of every object:

```julia
TraceCalls.store(x) = REPR(x)
```

This essentially turns TraceCalls.jl into a traditional, non-explorable tracing package, but it guarantees that each value is shown as it was when the call was made.

# Debugging with traces

compare_past_trace

In [5]:
Pkg.test("TraceCalls")

INFO: Computing test dependencies for TraceCalls...
INFO: No packages to install, update or remove
INFO: Testing TraceCalls


Error During Test
  Test threw an exception of type BoundsError
  Expression: (t[1])() == 6
  BoundsError: attempt to access 0-element Array{TraceCalls.Trace,1} at index [1]
  Stacktrace:
   [1] getindex(::TraceCalls.Trace, ::Int64) at /Users/cedric/.julia/v0.6/TraceCalls/src/TraceCalls.jl:68
   [2] include_from_node1(::String) at ./loading.jl:569
   [3] include(::String) at ./sysimg.jl:14
   [4] process_options(::Base.JLOptions) at ./client.jl:305
   [5] _start() at ./client.jl:371


ERROR: LoadError: There was an error during testing
while loading /Users/cedric/.julia/v0.6/TraceCalls/test/runtests.jl, in expression starting on line 29
=============================[ ERROR: TraceCalls ]==============================

failed process: Process(`/Applications/Julia-0.6.app/Contents/Resources/julia/bin/julia -Ccore2 -J/Applications/Julia-0.6.app/Contents/Resources/julia/lib/julia/sys.dylib --compile=yes --depwarn=yes --check-bounds=yes --code-coverage=none --color=yes --compilecache=yes /Users/cedric/.julia/v0.6/TraceCalls/test/runtests.jl`, ProcessExited(1)) [1]

INFO: No packages to install, update or remove


LoadError: [91mTraceCalls had test errors[39m

# Profiling